<a href="https://colab.research.google.com/github/EgeniyaKlochneva/ML-in-business/blob/HW3/HWL3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа к уроку 3. Связь бизнес-показателей и DS-метрик

**Домашнее задание**

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта

2. при обучении моделей обязательно использовать кроссвалидацию

3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

4. сделать выводы о том, какая модель справилась с задачей лучше других

5. (опциональный вопрос) какой график (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого, например, 1 к 1000).

**p.s.** В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

**Ссылки**
https://dyakonov.org/2017/07/28/auc-roc-площадь-под-кривой-ошибок/
https://en.wikipedia.org/wiki/Receiver_operating_characteristic

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [ ]:
# Разделим наши данные на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', axis=1), 
                                                    df['cardio'], random_state=0)

In [ ]:
# Преобразование
# К полям:
# gender, cholesterol применим OHE-кодирование
# age, height, weight, ap_hi, ap_lo - standardScaler
# gluc, smoke, alco, active - оставим пока как есть

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]


class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [
            col for col in pd.get_dummies(X, prefix=self.key).columns
        ]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer = Pipeline([('selector', NumberSelector(key=cont_col)),
                           ('standard', StandardScaler())])
    continuos_transformers.append((cont_col, transfomer))

for cat_col in cat_cols:
    cat_transformer = Pipeline([('selector', ColumnSelector(key=cat_col)),
                                ('ohe', OHEEncoder(key=cat_col))])
    cat_transformers.append((cat_col, cat_transformer))

for base_col in base_cols:
    base_transformer = Pipeline([('selector', NumberSelector(key=base_col))])
    base_transformers.append((base_col, base_transformer))

In [ ]:
# Объединим все трансформеры с помощью FeatureUnion
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [ ]:
# Добавим классификатор и запустим кросс-валидацию
log_reg = Pipeline([('features', feats),
                    ('classifier', LogisticRegression(random_state=42))])

In [ ]:
def scores(classifier):
    from sklearn.metrics import f1_score
    from sklearn.metrics import roc_auc_score

    #запустим кросс-валидацию
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)

    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]
    
    #рассчитаем метрики
    b = 1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
    roc_auc_score = roc_auc_score(y_test, y_score)
    ix = np.argmax(fscore)
    
 
    return [round(i, 3) for i in [cv_score, thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc_score]]

In [ ]:
%time

results = pd.DataFrame([
    scores(classifier=Pipeline([('features', feats), ('classifier', LogisticRegression(random_state=42))])),
    scores(classifier=Pipeline([('features', feats), ('classifier', RandomForestClassifier(random_state=42))])),
    scores(classifier=Pipeline([('features', feats), ('classifier', DecisionTreeClassifier(max_depth=4, random_state=42))])),
], columns=['CV_score', 'Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC score'])

results['models'] = ['LogisticRegression', 'RandomForestClassifier', 'DecisionTreeClassifier']
results = results.set_index('models')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.58 µs


In [ ]:
results

,CV_score,Best Threshold,F-Score,Precision,Recall,ROC AUC score
models,,,,,,
LogisticRegression,0.787,0.387,0.730,0.647,0.838,0.784
RandomForestClassifier,0.773,0.350,0.719,0.643,0.816,0.771
DecisionTreeClassifier,0.787,0.380,0.731,0.647,0.840,0.789


**Выводы:**
По ROC_AUC, наилучшей из представленных моделей является дерево решений.

1) Нужно максимизировать Recall, чтобы минимизировать случаи не распознавания болезни

2) Значения метрик не сильно отличаются и есть вероятность, что при других настройках лучшей будет другая модель